In [1]:
from load_portfolios import *
import os
import pandas as pd
import numpy as np

In [2]:
# the code to load portfolio
filename = './Daily/managed_portfolios_anom_d_55.csv'
dropna_perc = 0.2 # could be 1, which leads to 41; if 0.5, leads to 55 (all)
dates_anomaly, re_anomaly, mkt_anomaly, names_anomaly = load_managed_portfolios(filename, drop_perc=dropna_perc)

In [4]:
# print the first and last of dates_anomaly
print(dates_anomaly.iloc[0], dates_anomaly.iloc[-1])

1979-07-02 00:00:00 2019-12-31 00:00:00


#### The In-Sample Version (Replication)
- Full data, no train/test split, full data (no PCA transformation)

In [30]:
tT0 = '2019-01-01'
# to strftime
tT0 = pd.to_datetime(tT0)
freq = 252 # daily freq

re_anomaly.index = dates_anomaly.values
mkt_anomaly.index = dates_anomaly.values
mkt0 = mkt_anomaly.copy()

In [31]:
def demarket(r, mkt, b=None):
    if b is None:
        # Compute beta_i = Cov(r_i, mkt) / Var(mkt) for each asset
        cov = mkt.T @ r / len(mkt) - mkt.mean() * r.mean(axis=0)
        var = mkt.var()
        b = cov / var
    # Residuals
    rme = r - np.outer(mkt, b)
    return rme, b

In [32]:
r0, _ = demarket(re_anomaly.loc[:tT0, :], mkt0.loc[:tT0])
r0 = r0.divide(r0.std(axis=0), axis=1).multiply(mkt0.std())
T, n = r0.shape
y = np.mean(r0, axis=0)

In [ ]:
def regcov_inv(r):
    T, n = r.shape
    X = np.cov(r, rowvar=False)
    a = n / (n + T)
    X_reg = (1 - a) * X + a * (np.trace(X) / n) * np.eye(n)
    return X_reg, np.linalg.pinv(X_reg)

def get_penalty(kappa, T, X, freq):
    total_var = np.trace(X)
    return freq * total_var / (T * kappa**2)

In [38]:
def l2est(X, y, l):
    A = X + l * np.eye(X.shape[0])
    return np.linalg.solve(A, y)

def l2est_se(X, y, l):
    A = X + l * np.eye(X.shape[0])
    Ainv = np.linalg.inv(A)
    b = Ainv @ y
    se = np.sqrt(np.diag(Ainv) / X.shape[0])
    return b, se

In [41]:
X, Xinv = regcov_inv(r0)

In [47]:
lr = np.arange(1, 26)   # [1, 2, ..., 25]
lm = 1

z = np.column_stack([
    l2est(X, y, get_penalty(2**(i - lm), T, X, freq)) for i in lr
])

In [64]:
gridsize = 50
kfold = 3

In [65]:
# 1. Identify region where coefficients still change
unstable = np.mean(
    np.abs(z[:, 1:] - z[:, :-1]) / (1 + np.abs(z[:, :-1])),
    axis=0
) > 0.1
rlim = np.nonzero(unstable)[0]

# 2. Build log grid of kappa values in this region
kappa_grid = np.logspace(
    np.log10(2**rlim[-1]),   # last unstable point
    np.log10(0.01),          # lower bound
    gridsize,                
)

# 3. Convert to penalties
penalties = [get_penalty(k, T, X, freq) for k in kappa_grid]
penalties_cv = [p / (1 - 1/kfold) for p in penalties]

In [66]:
penalties_cv

[3.713669637018798e-06,
 4.8786171229373566e-06,
 6.408999011372507e-06,
 8.41944905548211e-06,
 1.106056066978201e-05,
 1.4530167178845613e-05,
 1.908816058683245e-05,
 2.507595887259785e-05,
 3.2942080014455935e-05,
 4.327573837523979e-05,
 5.685097999580986e-05,
 7.468466276552729e-05,
 9.811262449322152e-05,
 0.00012888974427278346,
 0.00016932139227252317,
 0.00022243611423752044,
 0.0002922124856937898,
 0.0003838771284422128,
 0.0005042962123646545,
 0.0006624897681124039,
 0.0008703073354361517,
 0.0011433155568154555,
 0.0015019642018772013,
 0.0019731179640414443,
 0.002592068769120608,
 0.0034051793285022726,
 0.004473355953126596,
 0.005876610760518945,
 0.007720055008479621,
 0.010141772488040057,
 0.013323162734746187,
 0.017502528819874286,
 0.0229929275195006,
 0.030205618934144696,
 0.03968087206037417,
 0.052128433815732005,
 0.06848069286246246,
 0.08996252048354512,
 0.11818301996457448,
 0.15525605699877754,
 0.20395859948436765,
 0.2679387272082552,
 0.35198889244